# Redes Sociais - APS 3

### Alunos: Arthur Barreto, Enricco Gemha e Felipe Catapano


Extraindo dados do *The Movies Dataset* [$^1$](https://www.kaggle.com/datasets/rounakbanik/the-movies-dataset), construímos uma rede de atores/atrizes em Hollywood que se conectam quando participam do mesmo filme como atores/atrizes, considerando mais de 45.000 filmes. Os vértices representam atores/atrizes em Hollywood e uma aresta não-direcionada indica um filme em que ambos atores/atrizes participaram.

## Pré-requisitos

In [ ]:
import graph_tool_extras as gte
import netpixi
from graph_tool import draw
import distribution as dst

In [ ]:
PATH = 'credits.csv'

## Análise dos dados importados

O arquivo relevante à análise é `credits.csv`, que representa uma lista de dicionários, cada um representando os dados de um personagem, que é consequentemente interpretado por um ator/atriz.

In [ ]:
with open(PATH) as file:

    # cria index de contagem para o loop abaixo.
    i = 0
    
    # ignora o cabeçalho.
    next(file)

    # Para não sobrecarregar este notebook
    # vamos espiar somente as 5 primeiras linhas.
    for line in file:

        # Transforma a linha em uma lista de partes,
        # considerando a vírgula como separador.
        parts = line.split(',')
        no1 = parts[0]
        no2 = parts[1].replace('\n', '')

        print(f'{no1} - {no2}')
        
        # Para não sobrecarregar este notebook, vamos usar um contador
        # e um break para imprimir apenas as cinco primeiras linhas.
        i += 1
        if i == 5:
            break